# 🚀 生产级学术术语提取系统 (Production-Grade Terminology Extraction)

**本代码旨在解决传统 NLP 提取中的三大痛点：**
1.  **噪音控制：** 自动剔除 `et al`, `Figure 1`, `DOI` 等非语义内容。
2.  **语法精准：** 使用 POS Pattern (词性模式) 替代粗糙的名词短语提取，只保留 `Adj+Noun` 或 `Noun+Noun`。
3.  **权重校准：** 修复 TF-IDF 计算逻辑，确保长尾专业术语能获得高分。

---

In [76]:
# === Cell 1: 环境配置 (中文适配版) ===
import sys
import os

# 1. 安装必要的库 (增加 jieba 用于辅助处理，虽然主要用 spaCy)
packages = [
    "pymupdf",       # PDF 解析
    "spacy",         # NLP 核心
    "scikit-learn",  # TF-IDF
    "pandas",        # 数据处理
    "openpyxl",      # Excel 导出
    "requests",      # 下载示例
    "jieba"          # 中文辅助分词
]

print("🛠️ 正在检查环境依赖...")
for pkg in packages:
    try:
        __import__(pkg)
    except ImportError:
        print(f"  ⬇️ 正在安装: {pkg}...")
        !{sys.executable} -m pip install {pkg} -q

# 2. 下载 spaCy 中文模型
import spacy
try:
    # 尝试加载中文模型
    nlp = spacy.load("zh_core_web_sm")
    print("✅ 中文 NLP 模型 (zh_core_web_sm) 已加载")
except OSError:
    print("⬇️ 未检测到中文模型，正在下载 (约 15MB)...")
    !{sys.executable} -m spacy download zh_core_web_sm
    nlp = spacy.load("zh_core_web_sm")
    print("✅ 中文 NLP 模型下载并加载完成")

print("🚀 环境准备就绪！")

🛠️ 正在检查环境依赖...
  ⬇️ 正在安装: scikit-learn...
✅ 中文 NLP 模型 (zh_core_web_sm) 已加载
🚀 环境准备就绪！


In [78]:
# === Cell 2: PDF 解析与中文排版修复 (自动搜寻版) ===
import fitz  # PyMuPDF
import re
import requests
import os
import glob  # 新增：用于查找文件

class ChinesePDFProcessor:
    @staticmethod
    def clean_text_structure(text):
        """
        专门修复中文 PDF 的排版问题
        """
        if not text: return ""
        
        # 1. 修复中文换行 (关键)：如果前一个字是中文，后一个字也是中文，中间的换行和空格删掉
        text = re.sub(r'([\u4e00-\u9fa5])\s*\n\s*([\u4e00-\u9fa5])', r'\1\2', text)
        
        # 2. 修复英文连字符 (Word- breaking) -> Wordbreaking
        text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
        
        # 3. 移除引用标记 [1], [1-3]
        text = re.sub(r'\[\d+(?:-\d+)?\]', '', text)
        
        # 4. 统一标点符号
        return text.strip()

    @staticmethod
    def parse_pdf(pdf_path):
        """解析 PDF，返回 (全文, 页面列表, 标题)"""
        if not os.path.exists(pdf_path):
            raise FileNotFoundError(f"❌ 文件未找到: {pdf_path}")
            
        doc = fitz.open(pdf_path)
        full_text = []
        pages_corpus = []
        
        # 尝试获取标题
        title = doc.metadata.get('title', '')
        if not title or "Untitled" in title or not title.strip():
            title = os.path.splitext(os.path.basename(pdf_path))[0]
        
        # 清洗标题中的非法字符
        title = re.sub(r'[\\/*?:"<>|]', '_', title)
        
        print(f"📖 正在解析: 《{title}》 (共 {len(doc)} 页)...")
        
        for page in doc:
            raw_text = page.get_text()
            cleaned_page = ChinesePDFProcessor.clean_text_structure(raw_text)
            
            if len(cleaned_page) > 50: # 忽略太短的页
                pages_corpus.append(cleaned_page)
                full_text.append(cleaned_page)
                
        return "".join(full_text), pages_corpus, title

# === 自动执行部分 (Auto-Run) ===

# 1. 自动查找当前目录下的所有 PDF
pdf_files = glob.glob("chinesepaper.pdf")

if not pdf_files:
    print("❌ 错误：当前目录下没找到任何 PDF 文件！")
    print("💡 请在左侧文件栏上传你的论文 PDF，然后重新运行此 Cell。")
    # 为了防止后续报错，设置空变量
    raw_text, raw_pages, doc_title = "", [], ""
else:
    # 2. 默认取第一个找到的 PDF
    target_pdf = pdf_files[0]
    print(f"✅ 检测到文件: {target_pdf}")
    
    try:
        # 3. 真正开始执行解析
        raw_text, raw_pages, doc_title = ChinesePDFProcessor.parse_pdf(target_pdf)
        print(f"🎉 解析成功！文本长度: {len(raw_text)} 字符")
        print("👉 请继续运行 Cell 3 和 Cell 4")
        
    except Exception as e:
        print(f"❌ 解析失败: {e}")
        raw_text, raw_pages, doc_title = "", [], ""

✅ 检测到文件: chinesepaper.pdf
📖 正在解析: 《chinesepaper》 (共 4 页)...
🎉 解析成功！文本长度: 5976 字符
👉 请继续运行 Cell 3 和 Cell 4


In [79]:
# === Cell 3: 中文术语提取核心引擎 (API 修复版) ===
import spacy
from spacy.matcher import Matcher
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pandas as pd
import jieba

class ChineseTermExtractor:
    def __init__(self, nlp_model):
        self.nlp = nlp_model
        self.matcher = Matcher(self.nlp.vocab)
        self.rejection_log = []
        
        # --- 1. 中文语法模式 ---
        self.matcher.add("TERM", [
            # 模式 A: 纯名词短语 (e.g., 神经网络)
            [{"POS": "NOUN"}, {"POS": "NOUN"}], 
            [{"POS": "NOUN"}, {"POS": "NOUN"}, {"POS": "NOUN"}],
            
            # 模式 B: 形容词 + 名词 (e.g., 深度学习)
            [{"POS": "ADJ"}, {"POS": "NOUN"}],
            [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}],
            
            # 模式 C: 专有名词 + 名词 (e.g., Transformer模型)
            [{"POS": "PROPN"}, {"POS": "NOUN"}],
            [{"POS": "NOUN"}, {"POS": "PROPN"}] 
        ])
        
        # --- 2. 中文语义狙击黑名单 ---
        self.blacklist = {
            # 论文结构
            "摘要", "引言", "绪论", "目录", "参考文献", "致谢", "附录", "图表",
            "第一章", "第二章", "第三章", "第四章", "第五章", "第六章",
            "图", "表", "公式", "代码", "数据来源", "关键词",
            
            # 学术套话
            "研究", "分析", "方法", "结果", "讨论", "结论", "数据", "实验",
            "应用", "基于", "使用", "提出", "利用", "采用", "对比", "比较",
            "旨在", "为了", "因为", "所以", "但是", "而且", "通过", "对于",
            "显著", "差异", "水平", "影响", "因素", "机制", "策略", "现状",
            "问题", "对策", "发展", "趋势", "理论", "模型", "体系", "框架",
            
            # 计量与时间
            "时间", "年份", "年度", "百分比", "平均值", "最大值", "最小值",
            "数量", "频率", "总数", "范围", "程度", "效率", "质量", "性能",
            "一月", "二月", "三月", "四月", "五月", "六月", "七月", "八月", "九月", "十月",
            "年", "月", "日", "周", "天", "小时", "分钟",
            
            # 出版信息
            "期刊", "杂志", "学报", "大学", "学院", "作者", "编辑", "出版",
            "基金", "项目", "资助", "收稿", "日期"
        }
        
        self.bad_fixes = {
            "各种", "大量", "许多", "不同", "主要", "重要", "核心", "关键", "基本",
            "相关", "有关", "上述", "如下", "该", "本", "某", "高", "低", "大", "小"
        }

    @staticmethod
    def clean_text(text):
        """清洗文本用于 TF-IDF (静态方法)"""
        # 仅保留中文字符、英文字母、数字
        if not text: return ""
        text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9]', ' ', text)
        return re.sub(r'\s+', ' ', text).strip()

    def validate_term(self, term):
        """验证术语有效性"""
        clean_t = term.strip()
        
        if len(clean_t) < 2: return False, "太短"
        if len(clean_t) > 15: return False, "太长"
        
        if re.match(r'^[0-9a-zA-Z\s]+$', clean_t) and len(clean_t) < 4:
            return False, "纯英文/数字过短"
            
        for bad in self.blacklist:
            if bad == clean_t: return False, f"黑名单词汇 ({bad})"
            
        for fix in self.bad_fixes:
            if clean_t.startswith(fix): return False, f"无效前缀 ({fix})"
            
        return True, "有效"

    # === 修复点 1: 方法名改为 extract_candidates ===
    def extract_candidates(self, full_text):
        self.rejection_log = []
        print("🔍 正在进行 NLP 模式匹配 (这可能需要一点时间)...")
        
        doc = self.nlp(full_text[:1000000])
        matches = self.matcher(doc)
        
        candidates = set()
        for _, start, end in matches:
            span = doc[start:end]
            term = span.text.strip()
            if re.search(r'[\u4e00-\u9fa5]', term):
                term = term.replace(" ", "")
            
            is_valid, reason = self.validate_term(term)
            
            if is_valid:
                candidates.add(term)
            else:
                self.rejection_log.append({"Term": term, "Reason": reason})
                
        return list(candidates)

    # === 修复点 2: 方法名改为 process_deduplication ===
    def process_deduplication(self, df):
        """
        中文子串去重逻辑
        """
        if df.empty: return df
        print("✂️ 执行智能子串去重...")
        
        terms = df.sort_values("Score", ascending=False).to_dict('records')
        to_drop = set()
        
        for i, short in enumerate(terms):
            for j, long in enumerate(terms):
                if i == j: continue
                s_txt = short['Term']
                l_txt = long['Term']
                
                # 如果短词是长词的子串 (e.g. '算法' in '遗传算法')
                if s_txt in l_txt:
                    # 如果短词分数没有显著高于长词 (宽容度 2.0 倍)
                    if short['Score'] < (long['Score'] * 2.0):
                        to_drop.add(s_txt)
                        
        df_clean = df[~df['Term'].isin(to_drop)]
        print(f"   去除冗余词汇: {len(df) - len(df_clean)} 个")
        return df_clean

    def compute_tfidf(self, pages, vocab):
        print("📊 计算 TF-IDF 权重...")
        
        def chinese_tokenizer(text):
            return jieba.lcut(text)

        vectorizer = TfidfVectorizer(
            vocabulary=vocab,
            tokenizer=chinese_tokenizer, 
            ngram_range=(1, 3),
            norm='l2'
        )
        try:
            X = vectorizer.fit_transform(pages)
            scores = X.sum(axis=0).A1
            return dict(zip(vectorizer.get_feature_names_out(), scores))
        except Exception as e:
            print(f"⚠️ TF-IDF 计算部分跳过: {e}")
            return {}

# 初始化引擎
extractor = ChineseTermExtractor(nlp)
print("✅ 中文核心提取引擎构建完成 (API已修复)")

✅ 中文核心提取引擎构建完成 (API已修复)


In [80]:
# === Cell 4: 运行管道 (Robust Fix) ===
import pandas as pd

def run_full_pipeline(text_data, pages_data, config):
    # 1. 提取 (调用 Cell 3 的 extract_candidates)
    if not hasattr(extractor, 'extract_candidates'):
        raise AttributeError("❌ 你的 Cell 3 版本过旧，请重新运行上一轮对话中提供的【修复版 Cell 3】")
        
    candidates = extractor.extract_candidates(text_data)
    print(f"✅ 初步识别候选词: {len(candidates)} 个")
    
    # 2. 计算权重
    tfidf_map = extractor.compute_tfidf(pages_data, candidates)
    
    # 3. 评分
    results = []
    print("⚖️ 正在综合评分...")
    
    for term in candidates:
        freq = text_data.count(term)
        if freq < config['min_freq']: continue
        
        tfidf = tfidf_map.get(term, 0)
        
        # 词长奖励
        len_bonus = 1.0 + (len(term) * 0.1) 
        if len(term) > 6: len_bonus = 1.5 
        
        # 综合分
        score = (freq * config['w_freq'] + tfidf * config['w_tfidf']) * len_bonus
        
        # 上下文提取
        idx = text_data.find(term)
        if idx > -1:
            start = max(0, idx - 20)
            end = min(len(text_data), idx + len(term) + 20)
            ctx = text_data[start:end].replace('\n', '')
        else:
            ctx = ""
        
        results.append({
            "Term": term,
            "Score": round(score, 2),
            "Freq": freq,
            "TF-IDF": round(tfidf, 2),
            "Context": "..." + ctx + "..."
        })
        
    df = pd.DataFrame(results)
    
    # 4. 去重
    if hasattr(extractor, 'process_deduplication'):
        df_final = extractor.process_deduplication(df)
    else:
        df_final = extractor.deduplicate(df) # 兼容旧版方法名
    
    return df_final.sort_values("Score", ascending=False)

# 权重配置
config = {
    "min_freq": 2,      
    "w_freq": 0.4,      
    "w_tfidf": 15.0     
}

# === 关键修复：变量名自动对齐 ===
# 自动检测 Cell 2 生成了哪个变量，并赋值给 target_text / target_pages
target_text = None
target_pages = None

# 情况 A: Cell 2 生成了 raw_text (自动搜寻版)
if 'raw_text' in locals() and raw_text:
    print("🔗 成功对接 Cell 2 数据 (raw_text)")
    target_text = raw_text
    target_pages = raw_pages

# 情况 B: Cell 2 生成了 full_text (旧版)
elif 'full_text' in locals() and full_text:
    print("🔗 成功对接 Cell 2 数据 (full_text)")
    target_text = full_text
    target_pages = pages_corpus if 'pages_corpus' in locals() else []

# === 执行 ===
if target_text and target_pages:
    try:
        df_result = run_full_pipeline(target_text, target_pages, config)
        print(f"\n🏆 最终提取术语: {len(df_result)} 个")
        display(df_result.head(2000).style.background_gradient(subset=['Score'], cmap='Oranges'))
    except Exception as e:
        print(f"\n❌ 运行出错: {e}")
        print("💡 提示: 如果报错 'AttributeError', 请确保你已经运行了最新的【Cell 3】")
else:
    print("⚠️ 未找到解析后的文本数据。请确认：")
    print("1. 你已经运行了 Cell 2")
    print("2. Cell 2 显示 '🎉 解析成功'")

🔗 成功对接 Cell 2 数据 (raw_text)
🔍 正在进行 NLP 模式匹配 (这可能需要一点时间)...
✅ 初步识别候选词: 611 个
📊 计算 TF-IDF 权重...
⚖️ 正在综合评分...
✂️ 执行智能子串去重...
   去除冗余词汇: 10 个

🏆 最终提取术语: 87 个


C:\10_Workspace\Python-course\Week1\venv\Lib\site-packages\sklearn\feature_extraction\text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Term,Score,Freq,TF-IDF,Context
55,改革开放,50.220000,20,1.860000,...5 · 编者按：2018 年，中国迎来改革开放40 周年。改革开放以来的40 年，是中...
28,研究成果,23.420000,10,0.850000,...其是翻译学术出版专家唐瑾编审的全力支持，研究成果被列入“十三五”国家重点图书出版规划项目...
93,翻译研究,21.280000,38,0.000000,...活动日益频繁、不断丰富的40 年，是中国翻译研究不断探索、快速发展的40 年，也是中国翻...
15,学术交流,19.910000,7,0.760000,...方翻译理论，到强化交流与反思，加强国内外学术交流，学习和探索研究方法，再到关注理论话语建...
91,人才培养,14.090000,4,0.560000,...传播研究成果、引领翻译研究方向，促进学科人才培养，为学术交流搭建了重要的平台，为翻译学科...
74,中国翻译,12.880000,23,0.000000,...放40 周年。改革开放以来的40 年，是中国翻译文化事业繁荣昌盛的40 年，是翻译活动...
6,学术期刊,11.730000,4,0.450000,...译研究的最新成果和发展趋势。同时国内翻译学术期刊和论著也受到外国学者的关注。对外开放拓...
44,浙江大学,11.730000,4,0.450000,...改革开放以来中国翻译研究的发展之路许钧 浙江大学改革开放以来，我国翻译研究从初始开放和借...
35,文化交流,11.320000,4,0.430000,...开放以来，随着翻译在中外交流，尤其是中外文化交流中的地位的凸显，翻译活动所涉及的一些根本...
1,学科建设,11.130000,3,0.450000,...得的显著成就，全面把握中国翻译研究与翻译学科建设的整体状况，在中国社会不断发展与进步、中...


In [75]:
# === Cell 5: 导出与审计 ===
# 1. 查看拦截日志 (可选)
def show_audit_log(extractor):
    log = pd.DataFrame(extractor.rejection_log)
    if not log.empty:
        print(f"🛑 审计日志: 共拦截 {len(log)} 个无效词")
        print(log['Reason'].value_counts().head())
        # display(log.head(5))

show_audit_log(extractor)

# 2. 导出结果
if 'df_result' in locals():
    # 自动文件名
    safe_title = re.sub(r'[\\/*?:"<>|]', '_', title) if 'title' in locals() else "Result"
    filename = f"术语表_{safe_title}.xlsx"
    
    # 整理列名
    export_df = df_result.rename(columns={
        "Term": "术语 (中文/英文)",
        "Score": "推荐分",
        "Freq": "词频",
        "Context": "原文语境"
    })
    export_df.insert(1, "人工翻译/备注", "") # 插入空列方便填空
    
    try:
        export_df.to_excel(filename, index=False)
        print(f"\n🎉 成功导出文件: {filename}")
    except Exception as e:
        print(f"❌ 导出失败: {e}")

🛑 审计日志: 共拦截 38 个无效词
Reason
无效前缀 (高)     9
无效前缀 (大)     6
无效前缀 (各种)    6
无效前缀 (基本)    5
无效前缀 (重要)    5
Name: count, dtype: int64

🎉 成功导出文件: 术语表_Result.xlsx
